In [1]:
import os 
os.chdir('../')

In [2]:
pwd

'/Users/hitam/Downloads/End2End_ChickenDiseaseClassification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir:Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [4]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [6]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassification import logger
from ChickenDiseaseClassification.utils.common import get_size


import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
class DataIngestion():
    def __init__(self,config=DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_path = Path(self.config.local_data_file)
            filename, header = request.urlretrieve(
                    url=self.config.source_URL,
                    filename=self.config.local_data_file
            )

            logger.info(f"{filename} downloaded! with following info: \n{header}")
        else:
            logger.info(f"File already exists at path: {self.config.local_data_file} with size: {get_size(Path(self.config.local_data_file))}")

        
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
                                                                                                            

    

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-01-25 13:20:01,671: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-25 13:20:01,681: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-25 13:20:01,684: INFO: common: created directory at: artifacts]
[2024-01-25 13:20:01,690: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-25 13:20:01,710: INFO: 1030927333: File already exists at path: artifacts/data_ingestion/chicken_data.zip with size: ~ 11345 KB]
